In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# %cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]


Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 362 (delta 76), reused 294 (delta 71), pack-reused 0 (from 0)
Receiving objects: 100% (362/362), 9.95 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/kaggle/working/LLaMA-Factory
assets/       evaluation/  MANIFEST.in     requirements.txt  tests/
CITATION.cff  examples/    pyproject.toml  scripts/
data/         LICENSE      README.md       setup.py
docker/       Makefile     README_zh.md    src/
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 883.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 389.1 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571

In [4]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-14 11:21:09.357390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752492069.539329      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752492069.593796      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
%cd /kaggle/working/LLaMA-Factory

/kaggle/working/LLaMA-Factory


In [6]:
# Cell: Updated dataset registry with validation set
import json

dataset_info = {
    "dolly_train": {
        "file_name": "/kaggle/input/databricks-dolly-15k/train_data.json",
        "ranking": False,
        "formatting": "alpaca",
        "columns": {"prompt": "instruction", "query": "input", "response": "label"},
    },
    "dolly_validation": {
        "file_name": "/kaggle/input/databricks-dolly-15k/validation_data.json",
        "ranking": False,
        "formatting": "alpaca",
        "columns": {"prompt": "instruction", "query": "input", "response": "label"},
    },
    "dolly_test": {
        "file_name": "/kaggle/input/databricks-dolly-15k/test_data.json",
        "ranking": False,
        "formatting": "alpaca",
        "columns": {"prompt": "instruction", "query": "input", "response": "label"},
    },
}

# Save updated dataset registry
with open("data/dataset_info.json", "w") as f:
    json.dump(dataset_info, f, indent=2)

print("Updated dataset registry with train/validation/test splits!")


Updated dataset registry with train/validation/test splits!


In [7]:
# Cell: Updated training config with validation
import yaml

training_config_with_validation = {
    "model_name_or_path": "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    "template": "llama3",
    # Dataset configuration - NOW WITH VALIDATION
    "dataset": "dolly_train",  # Training dataset
    "eval_dataset": "dolly_validation",  # Validation dataset
    "dataset_dir": "data",
    # Training configuration
    "stage": "sft",
    "do_train": True,
    "do_eval": True,  # Enable validation evaluation
    "finetuning_type": "lora",
    # LoRA configuration
    "lora_target": "all",
    "lora_rank": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    # Training parameters
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 2,
    "learning_rate": 2e-4,
    "num_train_epochs": 2,
    "lr_scheduler_type": "cosine",
    "warmup_steps": 100,
    # Evaluation settings
    "eval_strategy": "steps",
    "eval_steps": 250,
    "save_strategy": "steps",
    "save_steps": 250,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_loss",
    "greater_is_better": False,
    # Optimization
    "fp16": True,
    "max_grad_norm": 1.0,
    "quantization_bit": 8,
    # Logging
    "logging_steps": 50,
    "save_total_limit": 3,
    "output_dir": "saves/dolly-llama-lora",
    # Sequence lengths
    # "max_source_length": 512,
    # "max_target_length": 256,
    # "cutoff_len": 768,
    # Other settings
    "overwrite_output_dir": True,
    "plot_loss": True,
    "report_to": "none",
    "use_unsloth": True,
}

# Save updated config
with open("training_config_with_validation.yaml", "w") as f:
    yaml.dump(training_config_with_validation, f, default_flow_style=False)

print("Updated training configuration with validation!")


Updated training configuration with validation!


In [8]:
!llamafactory-cli train training_config_with_validation.yaml

2025-07-14 11:21:41.053454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752492101.076094     148 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752492101.083068     148 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[WARNING|2025-07-14 11:21:51] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-07-14 11:21:51] llamafactory.hparams.parser:410 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 54.7kB [00:00, 127MB/s]
tokenizer.json: 100%|██████████████████████| 17.2M/17.2M [00:01<00:00, 15.9MB/s]
special_tokens

In [9]:
import yaml

evaluation_config = {
    ### examples/train_lora/hcc_medical_eval.yaml
    ### model
    "model_name_or_path": "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    "adapter_name_or_path": "saves/dolly-llama-lora",
    ### method
    "finetuning_type": "lora",
    "stage": "sft",
    "do_predict": True,
    "eval_dataset": "dolly_test",  # Use the test dataset for evaluation
    "template": "qwen3",
    "output_dir": "saves/dolly-llama-lora/eval",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 8,
    "predict_with_generate": True,
    # "max_samples": 1000,
    "report_to": "none",
    "use_unsloth": True,
}


with open("evaluation_config_with_validation.yaml", "w") as f:
    yaml.dump(evaluation_config, f, default_flow_style=False)


In [10]:
!pip install rouge-chinese

In [11]:
!llamafactory-cli train evaluation_config_with_validation.yaml

2025-07-14 20:25:52.515324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752524752.539239     378 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752524752.546846     378 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[INFO|2025-07-14 20:26:02] llamafactory.hparams.parser:410 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2023] 2025-07-14 20:26:02,697 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-3B-Instruct-bnb-4bit/snapshots/bb1d317a108579fb40e646af8924a5e7ec5604b1/tokenizer.json
[INFO|tokenization_utils_ba